<a href="https://colab.research.google.com/github/AmeyaKI/multimodal-asr/blob/main/asr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')
import os

Mounted at /content/drive


In [ ]:
import torch
import numpy as np
from transformers import AutoProcessor, AutoModelForCTC
import librosa
import os

In [ ]:
# randomly selected audio
sample_path = "/content/drive/MyDrive/colab_notebooks/asr/sample.wav"

In [ ]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: True
GPU name: Tesla T4


In [ ]:
MODEL = 'nvidia/parakeet-ctc-1.1b' # train custom model ?
device = 'cuda'

processor = AutoProcessor.from_pretrained(MODEL)
model = AutoModelForCTC.from_pretrained(MODEL, dtype='auto')

model.eval()

preprocessor_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/962 [00:00<?, ?B/s]

🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py
🚨 Config not found for parakeet. You can manually add it to HARDCODED_CONFIG_FOR_MODELS in utils/auto_docstring.py


model.safetensors:   0%|          | 0.00/4.25G [00:00<?, ?B/s]

ParakeetForCTC(
  (encoder): ParakeetEncoder(
    (subsampling): ParakeetEncoderSubsamplingConv2D(
      (layers): ModuleList(
        (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        (3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (4): ReLU()
        (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        (6): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (7): ReLU()
      )
      (linear): Linear(in_features=2560, out_features=1024, bias=True)
    )
    (encode_positions): ParakeetEncoderRelPositionalEncoding()
    (layers): ModuleList(
      (0-41): 42 x ParakeetEncoderBlock(
        (feed_forward1): ParakeetEncoderFeedForward(
          (linear1): Linear(in_features=1024, out_features=4096, bias=True)
          (activation): SiLUActivation()
          (linear2): Linear(in_fe

In [ ]:
waveform, sample_rate = librosa.load(
    sample_path,
    sr=None,
    mono=True
)

In [ ]:
# resample to 16,000 hz
TARGET_SAMPLE_RATE = 16000
if sample_rate != TARGET_SAMPLE_RATE:
    waveform = librosa.resample(
        waveform,
        orig_sr = sample_rate,
        target_sr = TARGET_SAMPLE_RATE
    )
# normalizing audio amp
max_val = np.max(np.abs(waveform))
if max_val > 0:
    waveform /= max_val

In [ ]:
# calc model inputs
model_inputs = processor(
    waveform,
    sampling_rate = TARGET_SAMPLE_RATE,
    return_tensors = 'pt'
)
model_inputs = model_inputs.to(
    device = model.device,
    dtype = model.dtype
)

In [ ]:
# Verifying
print(model_inputs.input_features.dtype)
print(next(model.parameters()).dtype)

torch.bfloat16
torch.bfloat16


In [ ]:
# obtain logits
with torch.no_grad():
    # Ensure input_features are in bfloat16 to match model's dtype
    model_inputs.input_features = model_inputs.input_features.to(torch.bfloat16)
    logits = model(**model_inputs).logits
    # (batch size, time steps, vocab size)

In [ ]:
# decode and trnaslate logits to predicted text
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

text = transcription[0]
print(text)

hello this is chitali this is a test file for speech to text transcription


In [ ]:
# Accuracy: Character Error Rate